# Map layers to Underworld

Each layer corresponds to the region between two surfaces as defined in `layer_index.csv`. Each layer is assigned a "material index" which is used to map the thermal and hydraulic properties,

- $k_h$: hydraulic conductivity (m/s)
- $\phi$: porosity
- $k_T$: thermal conductivity (W/m/K)
- $H$: rate of heat production (W/m$^3$)
- $T_b$: lower temperature boundary condition

In [1]:
import numpy as np
import underworld as uw
import pandas as pd
import os
from scipy.interpolate import RegularGridInterpolator

%matplotlib inline

In [2]:
zfile_directory = "../Zipped zmap surfaces/Zipped zmap surfaces/"
geotiff_directory = "../data/GAB_surfaces/GeoTiff/"
numpy_directory = "../data/GAB_surfaces/NumPy/"
png_directory = "../data/GAB_surfaces/png/"
surface_filename = numpy_directory + "{:s}.npz"
simulation_directory = "../simulations/"

df_layers = pd.read_csv("../data/GAB_surfaces/layer_index.csv", index_col=0)
n_layers = df_layers.shape[0]

data_dir = "../Data/"
Tmin = 298.0
Tmax = 500.0
Nx, Ny, Nz = 20,20,50 # global size

# define bounding box
xmin, xmax, ymin, ymax = -955637.8812, 1034362.2443650428, 6342298.2975, 8922298.39436168
zmin, zmax = -8000.0, 1200.0

# resolution
dx, dy, dz = 20e3, 20e3, 50.
Nx, Ny, Nz = int((xmax-xmin)/dx), int((ymax-ymin)/dy), int((zmax-zmin)/dz)

print("number of elements in x,y,z {} | total number of elements = {}".format((Nx,Ny,Nz), Nx*Ny*Nz))

number of elements in x,y,z (99, 129, 184) | total number of elements = 2349864


## Set up the mesh

Initialise a Q1 finite element mesh and mesh variables.

In [3]:
deformedmesh = True
elementType = "Q1"
mesh = uw.mesh.FeMesh_Cartesian( elementType = (elementType), 
                                 elementRes  = (Nx,Ny,Nz), 
                                 minCoord    = (xmin,ymin,zmin), 
                                 maxCoord    = (xmax,ymax,zmax)) 

gwHydraulicHead            = mesh.add_variable( nodeDofCount=1 )
temperatureField           = mesh.add_variable( nodeDofCount=1 )
temperatureField0          = mesh.add_variable( nodeDofCount=1 )
velocityField              = mesh.add_variable( nodeDofCount=3 )
heatProductionField        = mesh.add_variable( nodeDofCount=1 )


coords = mesh.data

Xcoords = np.unique(coords[:,0])
Ycoords = np.unique(coords[:,1])
Zcoords = np.unique(coords[:,2])
nx, ny, nz = Xcoords.size, Ycoords.size, Zcoords.size

	Global element size: 99x129x184
	Local offset of rank 0: 0x0x0
	Local range of rank 0: 99x129x184


## Deform mesh to surface topography

We want to deform the $z$-axis spacing so that the surface of the mesh is draped over the topography.

In [4]:
with np.load(surface_filename.format(df_layers['Layer name'][0])) as npz:
    topo_interp = RegularGridInterpolator((npz['y'], npz['x']), np.flipud(npz['data']))
    
    
local_topography = topo_interp((mesh.data[:,1], mesh.data[:,0]))

# depth above which to deform
z_deform = zmin

with mesh.deform_mesh():
    zcube = coords[:,2].reshape(nz,ny,nx)
    zcube_norm = zcube.copy()
    zcube_norm -= z_deform
    zcube_norm /= zmax - z_deform
    zcube_mask = zcube_norm < 0
    
    # difference to add to existing z coordinates
    dzcube = zcube_norm * -(zmax - local_topography.reshape(zcube.shape))
    
    mesh.data[:,2] += dzcube.ravel()
    coords = mesh.data

## Set up the types of boundary conditions

Set the left, right and bottom walls such that flow cannot pass through them, only parallel.
In other words for groundwater head $h$:

$ \frac{\partial h}{\partial x}=0$ : left and right walls

$ \frac{\partial h}{\partial y}=0$ : bottom wall

This is only solvable if there is topography or a non-uniform upper hydraulic head boundary condition.

In [5]:
topWall = mesh.specialSets["MaxK_VertexSet"]
bottomWall = mesh.specialSets["MinK_VertexSet"]

gwPressureBC = uw.conditions.DirichletCondition( variable      = gwHydraulicHead, 
                                               indexSetsPerDof = ( topWall   ) )

temperatureBC = uw.conditions.DirichletCondition( variable        = temperatureField,
                                                  indexSetsPerDof = (topWall+bottomWall))

In [6]:
# create a linear gradient [0, 1] top to bottom of mesh
znorm = mesh.data[:,2].copy()
znorm -= zmin
znorm /= (zmax-zmin)
linear_gradient = 1.0 - znorm

# pressure and temperature initial conditions
initial_pressure = linear_gradient*(zmax-zmin)
initial_temperature = linear_gradient*(Tmax - Tmin) + Tmin
initial_temperature = np.clip(initial_temperature, Tmin, Tmax)

gwHydraulicHead.data[:]  = initial_pressure.reshape(-1,1)
temperatureField.data[:] = initial_temperature.reshape(-1,1)

# assign BCs (account for pressure of water below sea level)
sealevel = 0.0
seafloor = topWall[mesh.data[topWall,2] < sealevel]

gwHydraulicHead.data[topWall] = 0.
gwHydraulicHead.data[seafloor] = -((mesh.data[seafloor,2]-sealevel)*1.0).reshape(-1,1)
temperatureField.data[topWall] = Tmin
temperatureField.data[bottomWall] = Tmax

### Import water table surface

## Set up particle swarm

Each cell contains particles that _must_ be assigned isotropic thermal and hydraulic properties.

> __Four__ particles per cell seems to prevent the model from crashing.

In [7]:
gaussPointCount = 1
gaussPointPerCell = gaussPointCount**mesh.dim

swarm         = uw.swarm.Swarm( mesh=mesh )
swarmLayout   = uw.swarm.layouts.PerCellGaussLayout(swarm=swarm, gaussPointCount=gaussPointCount)
swarm.populate_using_layout( layout=swarmLayout )

In [8]:
materialIndex  = swarm.add_variable( dataType="int",    count=1 )
cellCentroid   = swarm.add_variable( dataType="double", count=3 )
swarmVelocity  = swarm.add_variable( dataType="double", count=3 )

hydraulicDiffusivity    = swarm.add_variable( dataType="double", count=1 )
fn_hydraulicDiffusivity = swarm.add_variable( dataType="double", count=1 )
thermalDiffusivity      = swarm.add_variable( dataType="double", count=1 )
heatProduction          = swarm.add_variable( dataType="double", count=1 )
a_exponent              = swarm.add_variable( dataType="double", count=1 )

In [9]:
# find the centroids using a single gaussPointCount per cell (parallel-safe, maybe)

swarm0         = uw.swarm.Swarm( mesh=mesh )
swarmLayout0   = uw.swarm.layouts.PerCellGaussLayout(swarm=swarm0, gaussPointCount=1)
swarm0.populate_using_layout( layout=swarmLayout0 )

cell_centroids = swarm0.data[:]
cellCentroid.data[:] = np.repeat(cell_centroids, gaussPointPerCell, axis=0)

## Import geological surfaces

Assign a material index for all cell centroids which lie between two surfaces

In [25]:
materialIndex.data[:] = 0
mask_layer = np.ones(swarm.data.shape[0], dtype=bool)

# starting from the surface and going deeper with each layer
for index in df_layers.index:
    row = df_layers.loc[index]
    
    # load surface
    with np.load(surface_filename.format(row['Layer name'])) as npz:
        layer_interp = RegularGridInterpolator((npz['y'], npz['x']), np.flipud(npz['data']))
        
    # interpolate surface to cell centroids
    z_interp = layer_interp((cellCentroid.data[:,1], cellCentroid.data[:,0]))
    
    # assign index to swarm particles which are below the current surface
    # if they are above the surface then we are done.
#     mask_layer[cellCentroid.data[:,2] > z_interp] = False
    mask_layer = cellCentroid.data[:,2] < z_interp
    materialIndex.data[mask_layer] = index
    
    print("Layer {:2d}  {}".format(index, row['Display name']))
    

Layer  0  Cenozoic
Layer  1  Late Cretaceous
Layer  2  Early Cretaceous
Layer  3  Early Cretaceous
Layer  4  Late Jurassic
Layer  5  Late Jurassic
Layer  6  Late Jurassic
Layer  7  Middle Jurassic
Layer  8  Early Jurassic
Layer  9  Early Jurassic
Layer 10  Late Triassic
Layer 11  Early Triassic
Layer 12  Late Permian
Layer 13  Early Permian


## Assign material properties

Use level sets to assign hydraulic diffusivities to a region on the mesh corresponding to any given material index.

- $H$       : rate of heat production
- $\rho$     : density
- $k_h$     : hydraulic conductivity
- $k_t$     : thermal conductivity
- $\kappa_h$ : hydraulic diffusivity
- $\kappa_t$ : thermal diffusivity


## Save to HDF5

In [26]:
xdmf_info_mesh  = mesh.save(simulation_directory+'mesh.h5')
xdmf_info_swarm = swarm.save(simulation_directory+'swarm.h5')

xdmf_info_matIndex = materialIndex.save(simulation_directory+'materialIndex.h5')
materialIndex.xdmf(simulation_directory+'materialIndex.xdmf', xdmf_info_matIndex,
                   'materialIndex', xdmf_info_swarm, 'TheSwarm')


# dummy mesh variable
phiField = mesh.add_variable( nodeDofCount=1 )


for xdmf_info,save_name,save_object in [(xdmf_info_swarm, 'materialIndexSwarm', materialIndex),
#                                         (xdmf_info_swarm, 'hydraulicDiffusivitySwarm', fn_hydraulicDiffusivity),
#                                         (xdmf_info_swarm, 'thermalDiffusivitySwarm', thermalDiffusivity),
#                                         (xdmf_info_swarm, 'heatProductionSwarm', heatProduction),
                                        ]:
    
    xdmf_info_var = save_object.save(simulation_directory+save_name+'.h5')
    save_object.xdmf(simulation_directory+save_name+'.xdmf', xdmf_info_var, save_name, xdmf_info, 'TheMesh')

    if save_name.endswith("Swarm"):
        # project swarm variables to the mesh
        hydproj = uw.utils.MeshVariable_Projection(phiField, save_object, swarm)
        hydproj.solve()

        field_name = save_name[:-5]+'Field'
        xdmf_info_var = phiField.save(simulation_directory+field_name+'.h5')
        phiField.xdmf(simulation_directory+field_name+'.xdmf',
                      xdmf_info_var, field_name,
                      xdmf_info_mesh, "TheMesh")